# Run Code

In [1]:
%matplotlib inline

import glob
import matplotlib.pyplot as plt
import nilearn.image, nilearn.masking, nilearn.plotting
import numpy as np
import pandas as pd
import scipy.stats
import seaborn as sns

from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

/Users/eyshin/.pyenv/versions/3.6.6/lib/python3.6/importlib/_bootstrap.py:219: ImportWarning: can't resolve package from __spec__ or __package__, falling back on __name__ and __path__
  return f(*args, **kwds)
/Users/eyshin/.pyenv/versions/3.6.6/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/Users/eyshin/.pyenv/versions/3.6.6/lib/python3.6/importlib/_bootstrap.py:219: ImportWarning: can't resolve package from __spec__ or __package__, falling back on __name__ and __path__
  return f(*args, **kwds)


In [2]:
data_dir = '/Volumes/clmnlab/GA/MVPA/LSS_pb02_MO_short_duration/data/'

In [3]:
roi_dir = '/Volumes/clmnlab/GA/fmri_data/masks/Fan/Fan280/'

roi_path = sorted(glob.glob(roi_dir + 'fan.roi.resam.*.nii'))
roi_imgs = []

for path in roi_path:
    roi_imgs.append(nilearn.image.load_img(path))

/Users/eyshin/.pyenv/versions/eyspy3.6/lib/python3.6/site-packages/nibabel/nifti1.py:582: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  ext_def = np.fromstring(ext_def, dtype=np.int32)


In [4]:
len(roi_path)

277

In [5]:
subj_list = [
    'GA01', 'GA02', 'GA05', 'GA07', 'GA08', 'GA11', 'GA12', 'GA13', 'GA14', 'GA15',
    'GA18', 'GA19', 'GA20', 'GA21', 'GA23', 'GA26', 'GA27', 'GA28', 'GA29', 'GA30',
    'GA31', 'GA32', 'GA33', 'GA34', 'GA35', 'GA36', 'GA37', 'GA38', 'GA42', 'GA44',
    'GB01', 'GB02', 'GB05', 'GB07', 'GB08', 'GB11', 'GB12', 'GB13', 'GB14', 'GB15',
    'GB18', 'GB19', 'GB20', 'GB21', 'GB23', 'GB26', 'GB27', 'GB28', 'GB29', 'GB30',
    'GB31', 'GB32', 'GB33', 'GB34', 'GB35', 'GB36', 'GB37', 'GB38', 'GB42', 'GB44'
]

In [6]:
data = {}

for subj in subj_list:
    for run in range(1, 7):
        data[subj, run] = nilearn.image.load_img(data_dir + 'betasLSS.MO.shortdur.%s.r%02d.nii.gz' % (subj, run))
    print(subj, end='\r')

/Users/eyshin/.pyenv/versions/eyspy3.6/lib/python3.6/site-packages/nibabel/nifti1.py:582: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  ext_def = np.fromstring(ext_def, dtype=np.int32)


In [7]:
for key, value in data.items():
    data[key] = nilearn.image.index_img(value, np.arange(1, 97))

In [8]:
target_path = list(range(1, 13)) * 8

In [9]:
target_pos = []

with open('/Volumes/clmnlab/GA/MVPA/LSS_pb02/behaviors/targetID.txt') as file:
    for line in file:
        target_pos.append(int(line.strip()))
        
target_pos = target_pos[1:97]

In [10]:
def fast_masking(img, roi):
    img_data = img.get_data()
    roi_mask = roi.get_data().astype(bool)
    
    if img_data.shape[:3] != roi_mask.shape:
        raise ValueError('different shape while masking! img=%s and roi=%s' % (img_data.shape, roi_mask.shape))
        
    return img_data[roi_mask, :].T

In [11]:
def accuracy(train_X, train_y, test_X, test_y):
    lda = LinearDiscriminantAnalysis(solver='lsqr', shrinkage='auto')
    lda.fit(train_X, train_y)
    
    return np.mean(lda.predict(test_X) == test_y)

def cross_validation(datasets, target):
    train_y = target + target
    test_y = target
    
    result = []
    
    for i in range(len(datasets)):
        train_X = np.concatenate([a for j, a in enumerate(datasets) if j != i])
        test_X = datasets[i]
        
        result.append(accuracy(train_X, train_y, test_X, test_y))
        
    return np.mean(result)

In [12]:
results = {}

for subj in subj_list:
    for m, roi in enumerate(roi_imgs):
        roi_index = m+1
        
        if ('pos', 'unpracticed', subj, roi_index) in results:
            continue
        
        X = [
            fast_masking(roi=roi, img=data[subj, 1]), 
            fast_masking(roi=roi, img=data[subj, 2]), 
            fast_masking(roi=roi, img=data[subj, 3]), 
        ]
        
        results['pos', 'practiced', subj, roi_index] = cross_validation(X, target_pos)
        
        X = [
            fast_masking(roi=roi, img=data[subj, 4]), 
            fast_masking(roi=roi, img=data[subj, 5]), 
            fast_masking(roi=roi, img=data[subj, 6]), 
        ]
        
        results['pos', 'unpracticed', subj, roi_index] = cross_validation(X, target_pos)
        
        print(subj, roi_index, end='\r')

/Users/eyshin/.pyenv/versions/eyspy3.6/lib/python3.6/site-packages/sklearn/discriminant_analysis.py:294: RuntimeWarning: internal gelsd driver lwork query error, required iwork dimension not returned. This is likely the result of LAPACK bug 0038, fixed in LAPACK 3.2.2 (released July 21, 2010). Falling back to 'gelss' driver.
  self.coef_ = linalg.lstsq(self.covariance_, self.means_.T)[0].T


In [14]:
len(results)

33240

In [13]:
import pickle

with open('20190715_p-and-unp_e-and-l_Fan.pkl', 'wb') as file:
    pickle.dump(file=file, obj=results)

# Visualization

In [ ]:
import pickle

with open('20190715_p-and-unp_e-and-l_Fan.pkl', 'rb') as file:
    wit_results = pickle.load(file=file)

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os, sys

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [ ]:
wit_df = pd.DataFrame(columns=['subj', 'roi_name', 'roi_idx', 'mean_acc', 'stage'])

for subj in subj_list[:30]:
    for i, roi_name in enumerate(names):
        wit_df = wit_df.append({'subj': subj,
                        'roi_name': roi_name,
                        'roi_idx': i+1,
                        'practiced': True,
                        'mean_acc': np.mean(wit_results['pos', 'practiced', subj, i+1]),
                        'stage': 'Early'}, ignore_index=True)
        
        wit_df = wit_df.append({'subj': subj,
                        'roi_name': roi_name,
                        'roi_idx': i+1,
                        'practiced': True,
                        'mean_acc': np.mean(wit_results['pos', 'practiced', subj.replace('A', 'B'), i+1]),
                        'stage': 'Late'}, ignore_index=True)
        
        wit_df = wit_df.append({'subj': subj,
                        'roi_name': roi_name,
                        'roi_idx': i+1,
                        'practiced': False,
                        'mean_acc': np.mean(wit_results['pos', 'unpracticed', subj, i+1]),
                        'stage': 'Early'}, ignore_index=True)
        
        wit_df = wit_df.append({'subj': subj,
                        'roi_name': roi_name,
                        'roi_idx': i+1,
                        'practiced': False,
                        'mean_acc': np.mean(wit_results['pos', 'unpracticed', subj.replace('A', 'B'), i+1]),
                        'stage': 'Late'}, ignore_index=True)

In [ ]:
import scipy.stats
import statsmodels.stats.multitest

pvals, tvals = [], []

for roi_name in names:
    sub_df = wit_df[(wit_df['roi_name'] == roi_name) & (wit_df['practiced'] == True)]
    
    early_mean_accs = sub_df[sub_df['stage'] == 'Early']['mean_acc']
    late_mean_accs = sub_df[sub_df['stage'] == 'Late']['mean_acc']
    
    res = scipy.stats.ttest_rel(early_mean_accs, late_mean_accs)
    pvals.append(res.pvalue)
    tvals.append(res.statistic)

res = statsmodels.stats.multitest.fdrcorrection(pvals)

for t, p, roi_name, uncorrected_p, tval in zip(*res, names, pvals, tvals):
    if uncorrected_p < 0.05:
        print('%21s'%roi_name, '%5s'%t, '%.2f'%p, '%5.2f'%tval, '%.3f'%uncorrected_p)

In [ ]:
import scipy.stats
import statsmodels.stats.multitest

pvals, tvals = [], []

for roi_name in names:
    sub_df = wit_df[(wit_df['roi_name'] == roi_name) & (wit_df['stage'] == 'Late')]
    
    early_mean_accs = sub_df[sub_df['practiced'] == True]['mean_acc']
    late_mean_accs = sub_df[sub_df['practiced'] == False]['mean_acc']
    
    res = scipy.stats.ttest_rel(early_mean_accs, late_mean_accs)
    pvals.append(res.pvalue)
    tvals.append(res.statistic)

res = statsmodels.stats.multitest.fdrcorrection(pvals)

for t, p, roi_name, uncorrected_p, tval in zip(*res, names, pvals, tvals):
    if uncorrected_p < 0.05:
        print('%21s'%roi_name, '%5s'%t, '%.2f'%p, '%5.2f'%tval, '%.3f'%uncorrected_p)

In [ ]:
wit_df['condition'] = ['%s, Practiced = %s' % (a, bool(b)) for a, b in zip(wit_df['stage'], wit_df['practiced'])]
wit_df.head()

In [ ]:
%config InlineBackend.figure_format = 'retina'

sns.set(rc={'figure.figsize':(10,200)})
sns.set(style="whitegrid")
plt.xlim(1/4, 0.75)
sns.barplot(data=wit_df, x='mean_acc', y='roi_name', hue='condition', orient='h', 
            capsize=0.1, ci=68)